In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /Users/mcabook/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mcabook/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [69]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [70]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [72]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

40 documents
12 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'negative', 'opentoday', 'payments', 'positive', 'problem: not flat', 'problem: shakes', 'rental', 'today']
74 unique lemmatized words ["'d", ',', 'a', 'accept', 'amount', 'anyone', 'are', 'awesome', 'axis', 'bearing', 'bye', 'can', 'card', 'cash', 'condition', 'cool', 'credit', 'cutting', 'day', 'do', 'doe', 'gib', 'good', 'goodbye', 'happened', 'have', 'hello', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'later', 'like', 'located', 'loose', 'machine', 'mastercard', 'moped', 'much', "n't", 'nice', 'nope', 'not', 'nothing', 'of', 'only', 'open', 'perfect', 'processing', 'rent', 'see', 'spindle', 'take', 'thanks', 'that', 'the', 'there', 'this', 'to', 'today', 'too', 'unsteadily', 'we', 'what', 'when', 'which', 'work', 'wrong', 'x', 'y', 'you', 'your']


In [73]:

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [74]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
8/8 [==============================] - 0s 2ms/step - loss: 2.5043 - accuracy: 0.0750
Epoch 2/200
8/8 [==============================] - 0s 2ms/step - loss: 2.4945 - accuracy: 0.0500
Epoch 3/200
8/8 [==============================] - 0s 2ms/step - loss: 2.4196 - accuracy: 0.0750
Epoch 4/200
8/8 [==============================] - 0s 2ms/step - loss: 2.3519 - accuracy: 0.2250
Epoch 5/200
8/8 [==============================] - 0s 2ms/step - loss: 2.3100 - accuracy: 0.2750
Epoch 6/200
8/8 [==============================] - 0s 2ms/step - loss: 2.2520 - accuracy: 0.3000
Epoch 7/200
8/8 [==============================] - 0s 2ms/step - loss: 2.1712 - accuracy: 0.3000
Epoch 8/200
8/8 [==============================] - 0s 2ms/step - loss: 2.0499 - accuracy: 0.4750
Epoch 9/200
8/8 [==============================] - 0s 2ms/step - loss: 2.0823 - accuracy: 0.3000
Epoch 10/200
8/8 [==============================] - 0s 3ms/step - loss: 1.9639 - accuracy: 0.4750
Epoch 11/200
8/8 [===========

8/8 [==============================] - 0s 2ms/step - loss: 0.1467 - accuracy: 0.9750
Epoch 85/200
8/8 [==============================] - 0s 2ms/step - loss: 0.1413 - accuracy: 0.9750
Epoch 86/200
8/8 [==============================] - 0s 2ms/step - loss: 0.1689 - accuracy: 0.9500
Epoch 87/200
8/8 [==============================] - 0s 1ms/step - loss: 0.1278 - accuracy: 0.9500
Epoch 88/200
8/8 [==============================] - 0s 2ms/step - loss: 0.1159 - accuracy: 1.0000
Epoch 89/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0936 - accuracy: 0.9750
Epoch 90/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0554 - accuracy: 1.0000
Epoch 91/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0980 - accuracy: 1.0000
Epoch 92/200
8/8 [==============================] - 0s 1ms/step - loss: 0.1569 - accuracy: 0.9500
Epoch 93/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0443 - accuracy: 1.0000
Epoch 94/200
8/8 [===============

8/8 [==============================] - 0s 2ms/step - loss: 0.0987 - accuracy: 0.9750
Epoch 168/200
8/8 [==============================] - 0s 1ms/step - loss: 0.1059 - accuracy: 0.9500
Epoch 169/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0158 - accuracy: 1.0000
Epoch 170/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0188 - accuracy: 1.0000
Epoch 171/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0428 - accuracy: 0.9750
Epoch 172/200
8/8 [==============================] - 0s 2ms/step - loss: 0.1200 - accuracy: 0.9500
Epoch 173/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0643 - accuracy: 0.9750
Epoch 174/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0692 - accuracy: 0.9750
Epoch 175/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0110 - accuracy: 1.0000
Epoch 176/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0548 - accuracy: 0.9500
Epoch 177/200
8/8 [=====

In [75]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [83]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# If it's related to a particular issue, then it returns a response corresponding 
# to the most likely solution based on the 'confidence score'; if it's a general question
# then choose from the response list randomly.


def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    if tag.startswith('problem:'):
        result = max(prob_sol_dict[tag], key = prob_sol_dict[tag].get)
        print(result)
        return result
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

#Matching the negative and positive feedback patterns and return "positive" or 'negative' as a string

def getFeedback(feedback, intents_json): # Returns Pos or Neg
    ints = predict_class(feedback, model)
    res = getResponse(ints, intents)
    print(ints,res)
    return res
        

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res,ints


In [91]:
#updates the priority of each possible solution by incrementing or substracting 1 from the total count

def updatePriority(solution,prob_tag, label):
    if prob_tag == '':
        return
    if label == 'positive':
        prob_sol_dict[prob_tag][solution] += 1
        print(prob_sol_dict[prob_tag])
    if label == 'negative':
        prob_sol_dict[prob_tag][solution] -= 1
        print(prob_sol_dict[prob_tag])

In [84]:
# A new data structure to store the multiple solutions with their frequency count(confidence score)
# {problem_tag1: {solution1: count1, solution2: count2,...},
#  problem _tag2: ...}

prob_sol_dict = {}

for item in intents['intents']:
    if item['tag'].startswith('problem:'):
        count_dict = {}
        for solution in item['responses']:
            count_dict[solution] = 1
        prob_sol_dict[item['tag']] = count_dict
prob_sol_dict[''] = 'puck'
print(prob_sol_dict)

In [95]:
#Creating GUI with tkinter

import tkinter
from tkinter import *
import tkinter.ttk as ttk

current_solution = ''
current_prob_tag = ''

def send():
    global current_solution
    global current_prob_tag

    
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res, probs_dict = chatbot_response(msg)
        tag = probs_dict[0]['intent']
        
        #Identifies problems from a message
        if tag.startswith('problem:'):
            print(tag)
            current_solution = res
            current_prob_tag = tag
            print(current_solution)
            
        # Identifies feedback from a message
        if tag == 'positive':
            print('---Feedback running---')
            updatePriority(current_solution, current_prob_tag, tag)
        if tag == 'negative':
            print('---Feedback running---')
            updatePriority(current_solution, current_prob_tag, tag)
            
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
ttk.Style().theme_use('alt')
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)


#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

Adjust the spindle bearing gaps
problem: not flat
Adjust the spindle bearing gaps
---Feedback running---
{'Adjust the spindle bearing gaps': 2, "Adjust the gib's gap": 1, 'wiggle your toes': 1, 'Have some vodka man': 1, 'Leave the company': 1}
---Feedback running---
{'Adjust the spindle bearing gaps': 1, "Adjust the gib's gap": 1, 'wiggle your toes': 1, 'Have some vodka man': 1, 'Leave the company': 1}
---Feedback running---
{'Adjust the spindle bearing gaps': 0, "Adjust the gib's gap": 1, 'wiggle your toes': 1, 'Have some vodka man': 1, 'Leave the company': 1}
Adjust the gib's gap
problem: not flat
Adjust the gib's gap
---Feedback running---
{'Adjust the spindle bearing gaps': 0, "Adjust the gib's gap": 0, 'wiggle your toes': 1, 'Have some vodka man': 1, 'Leave the company': 1}
wiggle your toes
problem: not flat
wiggle your toes
---Feedback running---
{'Adjust the spindle bearing gaps': 0, "Adjust the gib's gap": 0, 'wiggle your toes': 2, 'Have some vodka man': 1, 'Leave the company':